# Spark Kernel for getting stations and walking connections



### Start Spark

In [ ]:
%%configure
{"conf": {
    "spark.app.name": "datavirus_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6719,application_1589299642358_1210,pyspark,idle,Link,Link,
6726,application_1589299642358_1217,pyspark,idle,Link,Link,
6730,application_1589299642358_1221,pyspark,idle,Link,Link,
6733,application_1589299642358_1224,pyspark,idle,Link,Link,
6736,application_1589299642358_1227,pyspark,idle,Link,Link,
6740,application_1589299642358_1231,pyspark,idle,Link,Link,
6741,application_1589299642358_1232,pyspark,idle,Link,Link,
6742,application_1589299642358_1233,pyspark,idle,Link,Link,
6743,application_1589299642358_1234,pyspark,idle,Link,Link,
6744,application_1589299642358_1235,pyspark,idle,Link,Link,


In [2]:
# Initialization
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6763,application_1589299642358_1254,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import pyspark.sql.functions as F
import math

### Read the station list data from [BFKOORD_GEO](https://opentransportdata.swiss/en/cookbook/hafas-rohdaten-format-hrdf/#Abgrenzung)

In [5]:
metadata = spark.read.csv('/data/sbb/stations/bfkoordgeo.csv', header=True)
metadata.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+---------+------+----------------+
|StationID|Longitude| Latitude|Height|          Remark|
+---------+---------+---------+------+----------------+
|  0000002|26.074412|44.446770|     0|       Bucuresti|
|  0000003| 1.811446|50.901549|     0|          Calais|
|  0000004| 1.075329|51.284212|     0|      Canterbury|
|  0000005|-3.543547|50.729172|     0|          Exeter|
|  0000007| 9.733756|46.922368|   744|Fideris, Bahnhof|
+---------+---------+---------+------+----------------+
only showing top 5 rows

In [17]:
df_stations = metadata.withColumn("dlon", F.radians(F.col("Longitude")) - math.radians(8.540192)) \
             .withColumn("dlat", F.radians(F.col("Latitude")) - math.radians(47.378177)) \
             .withColumn("Distance_from_Zurich", F.asin(F.sqrt( F.sin(F.col("dlat") / 2) ** 2 + math.cos(math.radians(47.378177))
                                               *F.cos(F.radians(F.col("Latitude"))) * F.sin(F.col("dlon") / 2) ** 2)) * 2 * 3963 * 5280) \
             .drop("dlon", "dlat") \
             .filter(F.col("Distance_from_Zurich")<15000)

#zurich_stations = list(df_stations.select('Remark').toPandas()['Remark'])
df_stations.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

382

### Read the station list data from the time-table stops

In [3]:
stop_metadata = spark.read.orc("hdfs:///data/sbb/timetables/orc/stops")
stop_metadata.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+-------------+--------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+--------------------+----------------+----------------+-------------+--------------+
|1322000|            Altoggio|46.1672513851495|  8.345807131427|             |              |
|1322001|        Antronapiana| 46.060121674738|8.11361957990831|             |              |
|1322002|              Anzola|45.9898698225697|8.34571729989858|             |              |
|1322003|              Baceno|46.2614983591677|8.31925293162473|             |              |
|1322004|Beura Cardezza, c...|46.0790618438814|8.29927439970313|             |              |
+-------+--------------------+----------------+----------------+-------------+--------------+
only showing top 5 rows

In [13]:
df_stops = stop_metadata.withColumn("dlon", F.radians(F.col("stop_lon")) - math.radians(8.540192)) \
                 .withColumn("dlat", F.radians(F.col("stop_lat")) - math.radians(47.378177)) \
                 .withColumn("Distance_from_Zurich", F.asin(F.sqrt( F.sin(F.col("dlat") / 2) ** 2 + math.cos(math.radians(47.378177))
                                               *F.cos(F.radians(F.col("stop_lat"))) * F.sin(F.col("dlon") / 2) ** 2)) * 2 * 3963 * 5280) \
                 .drop("dlon", "dlat") \
                 .filter(F.col("Distance_from_Zurich")<15000)
df_stops.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

377

In [38]:
df_stops.where(F.col('stop_name')=='Zürich HB').show(5) #proof that there are many rows for the same station (for example, Zurich HB)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+---------+----------------+----------------+-------------+--------------+--------------------+
|        stop_id|stop_name|        stop_lat|        stop_lon|location_type|parent_station|Distance_from_Zurich|
+---------------+---------+----------------+----------------+-------------+--------------+--------------------+
|        8503000|Zürich HB|47.3781762039461|8.54019357578468|             |      8503000P|  0.4861881116144193|
|   8503000:0:10|Zürich HB|47.3794536181612|8.54019357578468|             |      8503000P|  466.22604663935687|
|   8503000:0:11|Zürich HB|47.3795144466376|8.54019357578468|             |      8503000P|   488.4408337792945|
|   8503000:0:12|Zürich HB|47.3786020121232|8.54019357578468|             |      8503000P|  155.21655831042875|
|   8503000:0:13|Zürich HB|47.3785411825942|8.54019357578468|             |      8503000P|   133.0014610545808|
|   8503000:0:14|Zürich HB|47.3783586935859|8.54019357578468|             |      8503000P|    66.3563436

### Compare both dataframes to see which stations are missing

In [40]:
only_names_stations = df_stations.select('StationID').distinct()
only_names_stops =  df_stops.where(F.length('stop_id')==7).select('stop_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
missing_ids = only_names_SBBdata.subtract(only_names_ORCdata)
missing_stations = df_stations.join(missing_ids,on =['StationID'])
missing_ids.show()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6790,application_1589299642358_1283,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'only_names_good' is not defined
Traceback (most recent call last):
NameError: name 'only_names_good' is not defined



### Create CSV file for storing the stations 

In [8]:
%%spark -o df_stations -n -1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
%%local
df_stations.to_csv("../data/Zurich_Stations.csv", index=False)

### Get the walking transfer times between different stations

In [18]:
station_pairs = df_stations.select('StationID','Longitude','Latitude','Height')
joinedDF = station_pairs.crossJoin(test).toDF('id1','lon1','lat1'.'h1','id2','lon2','lat2','h2')
joinedDF.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+---------+-------+--------+---------+
|    id1|    lon1|     lat1|    id2|    lon2|     lat2|
+-------+--------+---------+-------+--------+---------+
|0000176|8.521961|47.351679|0000176|8.521961|47.351679|
|0000176|8.521961|47.351679|8502572|8.513918|47.370293|
|0000176|8.521961|47.351679|8503000|8.540192|47.378177|
|0000176|8.521961|47.351679|8503001|8.488940|47.391481|
|0000176|8.521961|47.351679|8503003|8.548466|47.366611|
+-------+--------+---------+-------+--------+---------+
only showing top 5 rows

In [19]:
from pyspark.sql.types import IntegerType
distance = (
    joinedDF
        .withColumn("dlon", F.radians(F.col("lon1")) - F.radians(F.col("lon2")))
        .withColumn("dlat", F.radians(F.col("lat1")) - F.radians(F.col("lat2"))) 
        .withColumn("Distance", F.asin(F.sqrt( F.sin(F.col("dlat") / 2) ** 2 + F.cos(F.radians("lat2"))
                                           *F.cos(F.radians(F.col("lat1"))) * F.sin(F.col("dlon") / 2) ** 2)) * 2 * 3963 * 5280) \
        .drop("dlon", "dlat") 
        .filter(F.col("Distance")<500)
        .withColumn("Walking_time",F.round(60*(2+F.col("Distance")/50)).cast(IntegerType()))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
%%spark -o distance -n -1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
%%local
distance.to_csv("../data/Zurich_WalkingConnections_TOTAL.csv", index=False)

In [22]:
distance.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+---------+-------+--------+---------+------------------+------------+
|    id1|    lon1|     lat1|    id2|    lon2|     lat2|          Distance|Walking_time|
+-------+--------+---------+-------+--------+---------+------------------+------------+
|0000176|8.521961|47.351679|0000176|8.521961|47.351679|               0.0|         120|
|8502572|8.513918|47.370293|8502572|8.513918|47.370293|               0.0|         120|
|8503000|8.540192|47.378177|8503000|8.540192|47.378177|               0.0|         120|
|8503000|8.540192|47.378177|8503088|8.539170|47.377431|371.62272536447483|         566|
|8503000|8.540192|47.378177|8503446|8.541715|47.378846|448.94025446812276|         659|
+-------+--------+---------+-------+--------+---------+------------------+------------+
only showing top 5 rows